In [21]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
from keras.models import Sequential, load_model # type: ignore
from keras.layers import LSTM, Dense # type: ignore
from sklearn.preprocessing import MinMaxScaler # type: ignore
import pyodbc # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
from kerastuner.tuners import RandomSearch # type: ignore
from keras.callbacks import EarlyStopping # type: ignore
import matplotlib.pyplot as plt # type: ignore
import seaborn as sns # type: ignore
import os
from sklearn.metrics import mean_squared_error # type: ignore

In [22]:
def fetch_data():
    connection_string = "Driver={SQL Server Native Client 11.0};Server=MIRACLE\\MIRACOZTURK;Database=StokTahmin;uid=X;pwd=Y"
    try:
        connection = pyodbc.connect(connection_string)
        query = 'SELECT Tarih, StokKodu, SUM(Miktar) AS Miktar FROM StokTahmin.dbo.depo_stok GROUP BY Tarih, StokKodu ORDER BY Tarih DESC'
        df = pd.read_sql_query(query, connection)
    except pyodbc.Error as e:
        print("Veritabani hatasi:", e)
    except Exception as e:
        print("Sorgu calistirilirken bir hata olustu:", e)
    finally:
        if 'connection' in locals() and connection:
            connection.close()
    return df

In [ ]:
df = fetch_data()

In [ ]:
df.head(5)

In [ ]:
df.info()

In [26]:
scaler = MinMaxScaler()
df['Miktar_scaled'] = scaler.fit_transform(df[['Miktar']])

In [27]:
look_back = 30 # 30/60/90 gun yaklasimi.
features = []
labels = []

In [28]:
df['Tarih'] = pd.to_datetime(df['Tarih'])
df.sort_values(['StokKodu', 'Tarih'], inplace=True)

In [29]:
for i in range(look_back, len(df)-1):
    features.append(df['Miktar_scaled'].values[i-look_back:i])
    labels.append(df['Miktar_scaled'].values[i])

In [30]:
features = np.array(features)
labels = np.array(labels)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, shuffle=False)

In [32]:
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=256, step=32), input_shape=(look_back, 1)))
    model.add(Dense(1))
    model.compile(optimizer=hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd']), loss='mean_squared_error')
    return model

In [ ]:
def ensure_directories_exist(directory, project_name):
    full_path = os.path.join('C:\\Temp', directory, project_name)
    os.makedirs(full_path, exist_ok=True)  # exist_ok=True, dizin zaten varsa hata vermemesini saglar.

ensure_directories_exist('model_tuning', 'stock_prediction')

tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=3,
    directory='C:\\Temp\\model_tuning',
    project_name='stock_prediction'
)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[early_stopping])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"En iyi hiperparametreler: \n"
      f"Units: {best_hps.get('units')}\n"
      f"Optimizer: {best_hps.get('optimizer')}")

best_model = tuner.get_best_models(num_models=1)[0]


In [ ]:
trial_ids = []
units = []
optimizers = []
val_losses = []

for trial in tuner.oracle.trials.values():
    trial_ids.append(trial.trial_id)
    units.append(trial.hyperparameters.values['units'])
    optimizers.append(trial.hyperparameters.values['optimizer'])
    val_losses.append(trial.score)

results_df = pd.DataFrame({
    'trial_id': trial_ids,
    'units': units,
    'optimizer': optimizers,
    'val_loss': val_losses
})
print(results_df)


In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=results_df, x='units', y='val_loss', hue='optimizer', style='optimizer', s=100)
plt.title('Hiperparametrelerin Doğrulama Kaybina Göre Performansi')
plt.xlabel('Units')
plt.ylabel('Validation Loss')
plt.legend(title='Optimizer')
plt.grid(True)
plt.show()

In [ ]:
best_row = results_df.loc[results_df['val_loss'].idxmin()]
best_units = best_row['units']
best_optimizer = best_row['optimizer']
print(f"En iyi modelin parametreleri: \n{best_row}")

In [ ]:
history = best_model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[early_stopping])

plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Egitim Kaybi')
plt.plot(history.history['val_loss'], label='Dogrulama Kaybi')
plt.title('Model Kaybi')
plt.ylabel('Kayip')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
predictions = best_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print(f"Test seti uzerindeki MSE: {mse}")

In [ ]:
results = {}
best_model = tuner.get_best_models(num_models=1)[0]
for stock_code in df['StokKodu'].unique():
    data = df[df['StokKodu'] == stock_code]
    if len(data) >= look_back:
        last_sequence = data['Miktar_scaled'].values[-look_back:].reshape(1, look_back, 1)
        print("Boyut kontrolu - Last sequence shape:", last_sequence.shape)

        try:
            future_stock = best_model.predict(last_sequence)
            future_stock = scaler.inverse_transform(future_stock)
        except Exception as e:
            print(f"{stock_code} stok kodu icin tahminleme hatasi: {e}")
            continue

        last_date = data['Tarih'].iloc[-1]
        last_stock = data['Miktar'].iloc[-1]

        if last_stock == 0:
            results[stock_code] = "Stok zaten tukenmis."
        else:
            rate_of_depletion = future_stock[0, 0] / last_stock
            if np.isinf(rate_of_depletion):
                results[stock_code] = "Tahmin edilemiyor."
            else:
                days_to_deplete = int(np.ceil(rate_of_depletion))
                depletion_date = last_date + pd.Timedelta(days=days_to_deplete)
                results[stock_code] = depletion_date.strftime('%Y-%m-%d')
    else:
        results[stock_code] = "Yeterli veri yok."

In [ ]:
for stock_code, depletion_date in results.items():
    print(f"Stok Kodu {stock_code} icin stoklarin bitecegi tahmini tarih: {depletion_date}")